In [1]:
# imports básicos
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# Carregamento dos dados

In [2]:
base = pd.read_excel('dados/base_projeto_teste.xlsx',
                     usecols='A:O',   # usando as colunas de A:0      
                     index_col=0      # ID fixado como índice
                     )
base.head()

,Data Resposta,Marca,Marca (Outro Qual?),Satisfação Geral,Motivos da Satisfação,Expectativa Atendida,Qualidade Percebida,NPS,Gênero,Faixa Etária,Formação,Ocupação,Classe Econômica,UF
ID,,,,,,,,,,,,,,
1,2022-08-16 18:58:06,Marca 5,NaN,Satisfeito,Qualidade||Funcionalidade||Atendimento,Dentro das Expectativas,Próximo ao Ideal,9,Masculino,45 a 59 anos,Pós-graduação / Mestrado,Funcionário público,de 7.100 a 22.000,AL
2,2022-08-17 09:04:08,Marca 5,NaN,Nem satisfeito nem insatisfeito,Atendimento,Dentro das Expectativas,É um plano regular,5,Feminino,45 a 59 anos,Ensino superior completo,Empresário,De 2.900 a 7.100 reais,AL
3,2022-08-25 14:48:14,Marca 1,NaN,Satisfeito,Qualidade||Local||Funcionalidade,Dentro das Expectativas,Próximo ao Ideal,8,Masculino,25 a 34 anos,Pós-graduação / Mestrado,Funcionário público,de 7.100 a 22.000,AM
4,2022-08-17 17:42:31,Marca 4,NaN,Satisfeito,Local,Acima das Expectativas,Muito próximo do ideal,9,Masculino,35 a 44 anos,Ensino superior incompleto,Autônomo,Prefiro não responder,AM
5,2022-08-19 20:16:46,Outro (qual?),um,Muito Satisfeito,Local,Acima das Expectativas,Próximo ao Ideal,10,Masculino,45 a 59 anos,Ensino superior completo,Empresário,de 7.100 a 22.000,AM


Retirando espaços antes e depois dos nomes das colunas:

In [3]:
base.columns = [col.strip() for col in base.columns]
base.columns

Index(['Data Resposta', 'Marca', 'Marca (Outro Qual?)', 'Satisfação Geral',
       'Motivos da Satisfação', 'Expectativa Atendida', 'Qualidade Percebida',
       'NPS', 'Gênero', 'Faixa Etária', 'Formação', 'Ocupação',
       'Classe Econômica', 'UF'],
      dtype='object')

Reordenando o DataFrame pela ordem da coluna 'Data Resposta':

In [4]:
base.sort_values(by='Data Resposta', inplace=True)

A linha 46 é a única que tem dados referentes a setembro, de forma a não trazer padrões aos dados, desta forma optei por retira-la,

In [5]:
base.drop(46, inplace=True)

Desta forma, vamos ver como está a estrutura dos dados,

In [6]:
base.info()

<class 'pandas.core.frame.DataFrame'>
Index: 199 entries, 137 to 75
Data columns (total 14 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Data Resposta          199 non-null    datetime64[ns]
 1   Marca                  199 non-null    object        
 2   Marca (Outro Qual?)    67 non-null     object        
 3   Satisfação Geral       199 non-null    object        
 4   Motivos da Satisfação  199 non-null    object        
 5   Expectativa Atendida   199 non-null    object        
 6   Qualidade Percebida    199 non-null    object        
 7   NPS                    199 non-null    int64         
 8   Gênero                 199 non-null    object        
 9   Faixa Etária           199 non-null    object        
 10  Formação               199 non-null    object        
 11  Ocupação               199 non-null    object        
 12  Classe Econômica       199 non-null    object        
 13  UF       

Detalhes importantes,

- 'Data Resposta' já está no formato de data
- Apenas NPS é numérico.

Vamos separar dos dados para análises separadas.

# 1 - Tempo

In [7]:
base['Dia_de_semana'] = base['Data Resposta'].dt.day_name()   # convertendo os dias para dias de semana
base['Dia'] = base['Data Resposta'].astype(str).str[:10]      # separando apenas o dia XXXX-YY-ZZ 
base['Horario'] = base['Data Resposta'].astype(str).str[10:]  # separando apenas a hora XX:YY:ZZ

base[['Data Resposta', 'Dia_de_semana', 'Horario', 'Dia']].head()


,Data Resposta,Dia_de_semana,Horario,Dia
ID,,,,
137,2022-08-09 16:39:15,Tuesday,16:39:15,2022-08-09
138,2022-08-09 16:42:49,Tuesday,16:42:49,2022-08-09
185,2022-08-09 17:29:25,Tuesday,17:29:25,2022-08-09
160,2022-08-09 18:09:03,Tuesday,18:09:03,2022-08-09
147,2022-08-09 18:30:13,Tuesday,18:30:13,2022-08-09


# 2 - Satisfação

A coluna 'Motivos da Satisfação' tem muitos valores distindos, de forma a dificultar sua vizualização, assim, decidi trocar todos os valores que representam mais de um motivo, pela expressão 'Mais de um motivo',

In [8]:
base.loc[base['Motivos da Satisfação'].str.contains('||', regex=False), 'Motivos da Satisfação'] = 'Mais de um motivo'

base['Motivos da Satisfação'].value_counts()

Motivos da Satisfação
Mais de um motivo         127
Custo-Benefício            21
Local                      14
Funcionalidade             11
Nenhuma das anteriores      9
Qualidade                   7
Facilidade                  4
Atendimento                 4
Comunicação                 2
Name: count, dtype: int64

O mesmo problema de quantidade diferente de valores, se encontra nas colunas 'Satisfação Geral' e 'Expectativa Atendida', assim vamos redimensionar os dados para apenas 3 valores(classes) diferentes, para cada coluna,

In [9]:
base.loc[base['Satisfação Geral'].str.contains('Muito Satisfeito|Satisfeito'), 'Satisfação Geral'] = 'Positiva'
base.loc[base['Satisfação Geral'].str.contains('Muito Insatisfeito|Insatisfeito'), 'Satisfação Geral'] = 'Negativa'
base.loc[base['Satisfação Geral'].str.contains('Nem satisfeito nem insatisfeito'), 'Satisfação Geral'] = 'Neutra'
base['Satisfação Geral'].value_counts()

Satisfação Geral
Positiva    145
Neutra       36
Negativa     18
Name: count, dtype: int64

In [10]:
base.loc[base['Expectativa Atendida'].str.contains('Acima das Expectativas|Muito Acima das Expectativas '), 'Expectativa Atendida'] = 'Acima das Expectativas'
base.loc[base['Expectativa Atendida'].str.contains('Abaixo das Expectativas|Muito Abaixo das Expectativas'), 'Expectativa Atendida'] = 'Abaixo das Expectativas'
base.loc[base['Expectativa Atendida'].str.contains('Dentro'), 'Expectativa Atendida'] = 'Dentro das Expectativas'
base['Expectativa Atendida'].value_counts()

Expectativa Atendida
Dentro das Expectativas    131
Acima das Expectativas      36
Abaixo das Expectativas     32
Name: count, dtype: int64

- É importente notar que as classes 'Positiva' e 'Dentro das Expectativas' se destacam tendo mais que o *dobro* da soma dos outros valores.

# 3 - Marca

A coluna 'Marca (Outro Qual?)' e 'Marca' se completam, para tal tarefa vamos analisar a primeira coluna,

In [11]:
base['Marca (Outro Qual?)'] = base['Marca (Outro Qual?)'].astype(str) # transformando em str para ajudar a análise
base['Marca (Outro Qual?)'] 

ID
137                     nan
138                    nove
185        gosto da marca 2
160                   três 
147    Prefiro não informar
               ...         
6                       nan
55                      nan
33                      nan
86                      nan
75                      nan
Name: Marca (Outro Qual?), Length: 199, dtype: object

Existe uma grande quantidade de valores que representam a mesma classe, assim, vou usar a função *for* para agrupar tais valores,

In [12]:
lista = ['Prefiro não informar', 'um', 'dois|marca 2', 'três', 'quatro|marca 4', 'cinco', 'seis|6', 'sete|7', 'oito|8', 'nove|9', 'dez|10', 'onze', 'doze|12', 'treze|13', 'quatorze|14']
lista2 = ['Outros', 'Marca 1', 'Marca 2', 'Marca 3', 'Marca 4', 'Marca 5', 'Marca 6', 'Marca 7', 'Marca 8', 'Marca 9', 'Marca 10', 'Marca 11', 'Marca 12', 'Marca 13', 'Marca 14']

for x, y in zip(lista, lista2):
    base.loc[base['Marca (Outro Qual?)'].str.contains(x), 'Marca (Outro Qual?)'] = y

base.loc[base['Marca (Outro Qual?)'].str.contains('marca'), 'Marca (Outro Qual?)'] = 'Outros'

base.loc[base['Marca'].str.contains('Outro (qual?)', regex=False), 'Marca'] = base['Marca (Outro Qual?)']

base['Marca'].value_counts()

Marca
Marca 5     70
Marca 3     24
Marca 1     19
Outros      17
Marca 9     16
Marca 4     13
Marca 10     8
Marca 7      7
Marca 6      6
Marca 2      5
Marca 8      5
Marca 13     3
Marca 11     3
Marca 12     2
Marca 14     1
Name: count, dtype: int64

# 4 - Identificação do cliente

De forma parecida com os grupos anteriores, vamos reagrupar os valores,

In [13]:
base.loc[base['Faixa Etária'].str.contains('17 anos ou menos|18 a 24 anos|25 a 34 anos'), 'Faixa Etária'] = '17 a 34 anos'
base['Faixa Etária'].value_counts()

Faixa Etária
35 a 44 anos       65
45 a 59 anos       53
17 a 34 anos       48
60 anos ou mais    33
Name: count, dtype: int64

In [14]:
base.loc[base['Formação'].str.contains('Ensino superior incompleto|Ensino médio|Ensino fundamental'), 'Formação'] = 'Outros'
base['Formação'].value_counts()

Formação
Pós-graduação / Mestrado    76
Ensino superior completo    63
Outros                      60
Name: count, dtype: int64

# Base principal

Desta forma, a base de dados final, fica da seguinte forma,

In [15]:
base = base[['Marca', 'Dia_de_semana', 'Horario', 'Dia',
            'Satisfação Geral', 'Motivos da Satisfação', 
            'Expectativa Atendida', 'Qualidade Percebida', 'Gênero', 'Faixa Etária', 'Formação',
            'Ocupação', 'Classe Econômica', 'UF', 'NPS']]
base.head()

,Marca,Dia_de_semana,Horario,Dia,Satisfação Geral,Motivos da Satisfação,Expectativa Atendida,Qualidade Percebida,Gênero,Faixa Etária,Formação,Ocupação,Classe Econômica,UF,NPS
ID,,,,,,,,,,,,,,,
137,Marca 7,Tuesday,16:39:15,2022-08-09,Positiva,Custo-Benefício,Dentro das Expectativas,Próximo ao Ideal,Masculino,17 a 34 anos,Outros,Empregado CLT,De 2.900 a 7.100 reais,SP,9
138,Marca 9,Tuesday,16:42:49,2022-08-09,Positiva,Facilidade,Dentro das Expectativas,Muito próximo do ideal,Masculino,60 anos ou mais,Outros,Funcionário público,Até 2.900 reais,SP,10
185,Marca 2,Tuesday,17:29:25,2022-08-09,Positiva,Custo-Benefício,Dentro das Expectativas,É um plano regular,Feminino,45 a 59 anos,Ensino superior completo,Autônomo,Até 2.900 reais,SP,7
160,Marca 3,Tuesday,18:09:03,2022-08-09,Negativa,Mais de um motivo,Abaixo das Expectativas,É um plano regular,Feminino,45 a 59 anos,Pós-graduação / Mestrado,Prefiro não responder,Prefiro não responder,SP,3
147,Outros,Tuesday,18:30:13,2022-08-09,Positiva,Qualidade,Acima das Expectativas,Próximo ao Ideal,Feminino,45 a 59 anos,Ensino superior completo,Funcionário público,De 2.900 a 7.100 reais,SP,10


Por fim, salvando os dados em um arquivo '.xlsx',

In [16]:
base.to_excel('dados\\base_final.xlsx')